In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [2]:
def load_data(directory, label, img_size=(32, 32)):
    data = []
    labels = []
    for filename in os.listdir(directory):
        img_path = os.path.join(directory, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  
        img = cv2.resize(img, img_size)  
        data.append(img)
        labels.append(label)
    return np.array(data), np.array(labels)

In [3]:
closed_eye_path = r"C:\Users\sathy\DBA\model\Data\Closed"
open_eye_path = r"C:\Users\sathy\DBA\model\Data\Open"

In [4]:
open_eyes, open_labels = load_data(open_eye_path, label=0)
closed_eyes, closed_labels = load_data(closed_eye_path, label=1)

In [5]:
X = np.vstack((open_eyes, closed_eyes))
y = np.hstack((open_labels, closed_labels))

In [6]:
X = X / 255.0 
X = X.reshape(-1, 32, 32, 1)  
y = to_categorical(y, 2) 

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
from tensorflow.keras import layers, models

def build_lenet():
    model = models.Sequential([
        layers.Conv2D(6, kernel_size=(5, 5), activation='relu', input_shape=(32, 32, 1)),
        layers.AveragePooling2D(pool_size=(2, 2)),  # Added pool_size
        layers.Conv2D(16, kernel_size=(5, 5), activation='relu'),
        layers.AveragePooling2D(pool_size=(2, 2)),  # Added pool_size
        layers.Flatten(),
        layers.Dense(120, activation='relu'),
        layers.Dense(84, activation='relu'),
        layers.Dense(2, activation='softmax')  # 2 classes (open or closed)
    ])
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Build the model
model = build_lenet()


c:\Users\sathy\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6831 - loss: 0.6090 - val_accuracy: 0.8591 - val_loss: 0.3718
Epoch 2/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8850 - loss: 0.3308 - val_accuracy: 0.8625 - val_loss: 0.3629
Epoch 3/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8990 - loss: 0.2618 - val_accuracy: 0.9210 - val_loss: 0.2862
Epoch 4/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9327 - loss: 0.2081 - val_accuracy: 0.9244 - val_loss: 0.2371
Epoch 5/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9327 - loss: 0.1764 - val_accuracy: 0.9381 - val_loss: 0.2204
Epoch 6/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9555 - loss: 0.1371 - val_accuracy: 0.9450 - val_loss: 0.1759
Epoch 7/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9649 - loss: 0.0997 - val_accuracy: 0.9450 - val_loss: 0.1620
Epoch 8/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9672 - loss: 0.0897 - val_accuracy: 0.9485 - val_loss:

In [10]:
model.save("drowsiness_lenet_model.h5")

In [17]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9959 - loss: 0.0165 
Test Accuracy: 0.9907
